In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
from glob import glob

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import rasterio as rio

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
# import the necessary packages
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

In [4]:
import glob
import numpy as np
# import csv
from itertools import compress
import rasterio
import random
import json
import math
import time
import os
import sys
import pandas as pd
# Self written functions
#from helpers.helpers_read_data import *
# from helpers.helpers_models_tf2 import *
# from helpers.helpers_performance_analytics import *

In [43]:
images_path='./NorthCarolina_Sentinel2_PS_Data_scale30'
S_sentinel_bands=glob.glob(images_path+'/'+'*.tif')
S_sentinel_bands.sort()

l = []

for i in S_sentinel_bands:
  with rio.open(i, 'r') as f:
    l.append(f.read()[:,2:98,2:98])
l=np.moveaxis(l, 1,3)
arr_st = np.stack(l)

#arr_st

In [44]:
S_sentinel_bands=glob.glob(images_path+'/'+'*.tif')
S_sentinel_bands.sort()

image_ps_id = []

for i in S_sentinel_bands:
    image_ps_id.append(i.replace((images_path+'/'), '').replace('.tif', ''))

In [45]:
images_data_dict = dict(zip(image_ps_id,arr_st))

In [46]:
images_data_dict.keys()

dict_keys(['0', '1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '101', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '102', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1028', '1029', '103', '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '104', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '106', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '1069', '107', '1070', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '1078', '1079', '108', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '109', '1090', '1091', '1092', '1093', '1094', '1095', '1096', '1097', '1098', '1099', '11', '110', '1100', '1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108', '1109', '111', '11

In [47]:
dat_ps = pd.read_csv('./turnout_NC_nov3_2020_pres.csv')

In [48]:
dat_ps.head()

,countyXprecinct,county_name,precinct_name.x,polling_place_name,street_name,house_num,city,state,zip,vote_ct_agg,voters_registered,turnout
0,ALAMANCE ALBRIGHT,ALAMANCE,ALBRIGHT,SOUTHERN ALAMANCE HIGH SCHOOL,SOUTHERN HIGH SCHOOL RD,631,GRAHAM,NC,27253,2617,3128,0.836637
1,ALAMANCE BOONE 5,ALAMANCE,BOONE 5,ELON ELEMENTARY SCHOOL,WALKER RD,720,ELON,NC,27244,3335,4586,0.727213
2,ALAMANCE BURLINGTON 10,ALAMANCE,BURLINGTON 10,ST PAUL'S UNITED METHODIST CHU,TRAIL TWO,609,BURLINGTON,NC,27215,1425,2006,0.710369
3,ALAMANCE BURLINGTON 4,ALAMANCE,BURLINGTON 4,HILLCREST ELEMENTARY SCHOOL,DAVIS ST,1714,BURLINGTON,NC,27215,1454,1821,0.798462
4,ALAMANCE BURLINGTON 5,ALAMANCE,BURLINGTON 5,GROVE PARK BAPTIST CHURCH,TRAIL ONE,108,BURLINGTON,NC,27215,1643,2030,0.809360


In [49]:
#dat_ps.index.name = 'polling_place_name'
#dat_ps.reset_index(inplace=True)
# replace the / for a - to be in sync with the image names
#dat_ps['polling_place_name'] = dat_ps['polling_place_name'].str.replace('/','-')

In [50]:
dat_ps = pd.get_dummies(dat_ps, columns=['county_name'], prefix='', prefix_sep='')

In [51]:
dat_ps = dat_ps.dropna()

In [52]:
dat_ps.shape

(2545, 110)

In [53]:
dat_ps.head()
max(dat_ps['turnout']),min(dat_ps['turnout'])


(0.930327868852459, 0.261272650653182)

In [54]:
dat_ps.shape[0]

2545

In [55]:
len(images_data_dict.keys())

2545

In [56]:
images_data_dict['0'].shape

(96, 96, 3)

#### Baseline Model:

In [121]:
# TODO batch normalisation
inputs_img = keras.Input(shape=(96,96,3))
x = Conv2D(32, kernel_size=(3,3),padding='same', activation='relu')(inputs_img)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization(axis=-1)(x) 
x = Conv2D(64, kernel_size=(3,3),padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization(axis=-1)(x) 
# x = Conv2D(128, kernel_size=(3,3),padding='same', activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(256, kernel_size=(3,3),padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = BatchNormalization(axis=-1)(x) 
x = Flatten()(x)
x = Dense(8, activation = 'relu')(x)


inputs_ps_dat = keras.Input(shape=(99,))
y = Dense(64, activation="relu")(inputs_ps_dat)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="linear")(y)

combined_input = concatenate([x, y])

# trial
z = Dense(4, activation="relu")(combined_input)
z = Dense(1, activation="sigmoid")(z)
toy_model = Model(inputs=[inputs_img, inputs_ps_dat], outputs=z)

print(toy_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 96, 96, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 48, 48, 32)   0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 48, 48, 32)   128         max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [122]:
ps_id_at_ps_level=dat_ps.index

In [123]:
images_at_ps_level_wona = []
mask = [True]*len(ps_id_at_ps_level)

In [124]:
for i in ps_id_at_ps_level:
    try:
        images_at_ps_level_wona.append(images_data_dict[str(i)])
    except:
        mask[i]=False

In [125]:
images_at_ps_level_wona[0].shape

(96, 96, 3)

In [126]:
dat_ps_wona = dat_ps[mask]

In [127]:
print('We do not have data for',len(ps_id_at_ps_level) - len(images_at_ps_level_wona),'sat images. \n')

We do not have data for 1 sat images. 



In [128]:
dat_ps_wona.columns[:15]

Index(['countyXprecinct', 'precinct_name.x', 'polling_place_name',
       'street_name', 'house_num', 'city', 'state', 'zip', 'vote_ct_agg',
       'voters_registered', 'turnout', 'ALAMANCE', 'ALEXANDER', 'ALLEGHANY',
       'ANSON'],
      dtype='object')

In [129]:
# this generates the data we need for the model
dat_ps_features_wona = dat_ps_wona.copy()
turnout_at_ps_level_wona = list(dat_ps_features_wona.pop('turnout'))
ps_id_at_ps_level_wona = ps_id_at_ps_level[mask]

In [130]:
ps_id_at_ps_level_wona.shape

(2544,)

In [131]:
len(turnout_at_ps_level_wona)

2544

In [132]:
len(images_at_ps_level_wona)

2544

In [133]:
turnout_count_at_ps_level_wona = list(dat_ps_features_wona.pop('vote_ct_agg'))
registered_voters_at_ps_level_wona = list(dat_ps_features_wona.pop('voters_registered'))

In [134]:
dat_ps_features_wona.columns[7:]

Index(['zip', 'ALAMANCE', 'ALEXANDER', 'ALLEGHANY', 'ANSON', 'ASHE', 'AVERY',
       'BEAUFORT', 'BERTIE', 'BLADEN', 'BRUNSWICK', 'BUNCOMBE', 'BURKE',
       'CABARRUS', 'CALDWELL', 'CAMDEN', 'CARTERET', 'CASWELL', 'CATAWBA',
       'CHATHAM', 'CHEROKEE', 'CHOWAN', 'CLAY', 'CLEVELAND', 'COLUMBUS',
       'CRAVEN', 'CUMBERLAND', 'CURRITUCK', 'DARE', 'DAVIDSON', 'DAVIE',
       'DUPLIN', 'DURHAM', 'EDGECOMBE', 'FORSYTH', 'FRANKLIN', 'GASTON',
       'GATES', 'GRAHAM', 'GRANVILLE', 'GREENE', 'GUILFORD', 'HALIFAX',
       'HARNETT', 'HAYWOOD', 'HENDERSON', 'HERTFORD', 'HOKE', 'HYDE',
       'IREDELL', 'JACKSON', 'JOHNSTON', 'JONES', 'LEE', 'LENOIR', 'LINCOLN',
       'MACON', 'MADISON', 'MARTIN', 'MCDOWELL', 'MECKLENBURG', 'MITCHELL',
       'MONTGOMERY', 'MOORE', 'NASH', 'NEW HANOVER', 'NORTHAMPTON', 'ONSLOW',
       'ORANGE', 'PAMLICO', 'PASQUOTANK', 'PENDER', 'PERQUIMANS', 'PERSON',
       'PITT', 'POLK', 'RANDOLPH', 'RICHMOND', 'ROBESON', 'ROCKINGHAM',
       'ROWAN', 'RUTHERFORD', 'SA

In [135]:
dat_ps_features_wona=dat_ps_features_wona[dat_ps_features_wona.columns[8:]]

In [136]:
assert (dat_ps_features_wona.shape[0] == len(ps_id_at_ps_level_wona) == 
        len(images_at_ps_level_wona) == len(turnout_at_ps_level_wona)), 'Something went wrong in the data generation. There are different lengths'

In [137]:
# defines the new random order
new_order = random.sample(range(len(turnout_at_ps_level_wona)), len(turnout_at_ps_level_wona))

In [138]:
# defines train-test-val size
observed_ps_set_size = len(turnout_at_ps_level_wona)
testval_set_size = math.floor(.1*observed_ps_set_size)
# train_set_size = observed_ps_set_size - 2*testval_set_size
train_set_size = observed_ps_set_size - testval_set_size

In [139]:
# reshape images
image_data_all_bands = np.stack(images_at_ps_level_wona, axis = 0)

In [140]:
image_data_all_bands = image_data_all_bands.reshape((observed_ps_set_size, 96, 96, 3))

In [141]:
train_images = image_data_all_bands[new_order[0:train_set_size],:,:,:].reshape(train_set_size, 96, 96, 3)
# split in this way because validation taken in on the fly in model.fit()
# val_images = image_data_all_bands[new_order[train_set_size:(train_set_size+testval_set_size)],:,:,:].reshape((testval_set_size, 96, 96, 7))
# test_images = image_data_all_bands[new_order[(train_set_size+testval_set_size):observed_ps_set_size],:,:,:].reshape((testval_set_size, 96, 96, 7))
test_images = image_data_all_bands[new_order[train_set_size:observed_ps_set_size],:,:,:].reshape((testval_set_size, 96, 96, 3))

In [142]:
train_ps_data = np.array(dat_ps_features_wona)[new_order[0:train_set_size],:]
# val_ps_data = np.array(dat_ps_features_wona)[new_order[train_set_size:(train_set_size+testval_set_size)],:]
# test_ps_data = np.array(dat_ps_features_wona)[new_order[(train_set_size+testval_set_size):observed_ps_set_size],:]
test_ps_data = np.array(dat_ps_features_wona)[new_order[train_set_size:observed_ps_set_size],:]

In [143]:
# normalise the data
normalizer = Normalization()
normalizer.adapt(np.array(train_ps_data))

In [144]:
train_ps_data_normalized =normalizer(train_ps_data)
# val_ps_data_normalized =normalizer(val_ps_data)
# test_ps_data_normalized =normalizer(test_ps_data)

In [145]:
train_labels = np.array([turnout_at_ps_level_wona[i] for i in new_order[0:train_set_size]])
# val_labels = [turnout_at_ps_level_wona[i] for i in new_order[train_set_size:(train_set_size+testval_set_size)]]
# test_labels = np.array([turnout_at_ps_level_wona[i] for i in new_order[(train_set_size+testval_set_size):observed_ps_set_size]])
test_labels = np.array([turnout_at_ps_level_wona[i] for i in new_order[train_set_size:observed_ps_set_size]])

In [146]:
# Assert labels and data have same length
assert(train_ps_data.shape[0] == len(train_labels))
# assert(val_ps_data.shape[0] == len(val_labels))
assert(test_ps_data.shape[0] == len(test_labels))

In [147]:
# save the ids
# e.g. to write the train_val_test split as json for eventual later use
train_ps_id = [ps_id_at_ps_level_wona[i] for i in new_order[0:train_set_size]]
# val_ps_id = [ps_id_at_ps_level_wona[i] for i in new_order[train_set_size:(train_set_size+testval_set_size)]]
test_ps_id = [ps_id_at_ps_level_wona[i] for i in new_order[train_set_size:observed_ps_set_size]]

In [148]:
# set paths
best_model_path = './best_models/'
log_dir = './logs/'

In [149]:
# define callbacks
# select best models, but with early stopping
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=4,
    mode='min', min_delta=0.0001)
# save best model for fit later
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_path, monitor='val_mae', verbose=1,
    save_best_only=True, mode='min')
# tensorboard callbacks for debugging
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [early_stopper, checkpoint, tensorboard_callback]

In [150]:
# from https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
# opt = Adam(lr=1e-3, decay=1e-3 / 200)
opt = Adam()
# opt = RMSprop()
toy_model.compile(loss='mse',
             metrics=['mae'],
             optimizer=opt)

In [152]:
history = toy_model.fit(
    x=[train_images, train_ps_data_normalized], y=train_labels,
    validation_split=0.2, callbacks=callbacks_list,
    epochs=15, batch_size=64)

Epoch 1/15
29/29 [==============================] - 18s 599ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan

Epoch 00001: val_mae did not improve from inf


InvalidArgumentError: Nan in summary histogram for: conv2d_3/kernel_0 [Op:WriteHistogramSummary]